In [1]:
# import os
import sys

sys.path.insert(0, r"/mnt/c/Users/Trez/Desktop/tudat-bundle/tudatpy/")
import numpy as np
import datetime

from tudatpy.kernel.interface import spice
from tudatpy.kernel import numerical_simulation
from tudatpy.kernel.numerical_simulation import environment_setup
from tudatpy.kernel.numerical_simulation import propagation_setup
from tudatpy.kernel.numerical_simulation import estimation, estimation_setup
from tudatpy.kernel.numerical_simulation.estimation_setup import observation

# from astropy.time import Time
from astropy.coordinates import EarthLocation
from astropy import units as u

# from pac.data import BatchMPC
from tudatpy.data.mpc import BatchMPC
# import tudatpy.data as data

import matplotlib.pyplot as plt



In [2]:
# SPICE KERNELS
spice.load_standard_kernels()
# spice.load_kernel(r"codes_300ast_20100725.bsp")
# spice.load_kernel(r"codes_300ast_20100725.tf")

In [3]:
codes = [433, 1, 4]

batch = BatchMPC()
batch.get_observations(codes)
batch.filter(
    epoch_start=datetime.datetime(2018, 6, 1),
    epoch_end=datetime.datetime(2023, 6, 1),
    observatories_exclude=["C51", "C59", "C57"],
)

batch.summary()

print(batch.observatories_table(only_in_batch=True, only_space_telescopes=True, include_positions=False))


   Batch Summary:
1. Batch includes 3 minor planets:
   ['433', '1', '4']
2. Batch includes 1602 observations, including 0 observations from space telescopes
3. The observations range from 2018-06-12 10:39:16.416012 to 2023-05-29 11:10:29.827215
   In seconds TDB since J2000: 582072025.60063 to 738630699.0121946
   In Julian Days: 2458281.94394 to 2460093.965623
4. The batch contains observations from 68 observatories, including 0 space telescopes

Empty DataFrame
Columns: [Code, Name, count]
Index: []


In [4]:
bodies_to_propagate = batch.MPC_objects
central_bodies = ["Sun" for _ in batch.MPC_objects]
bodies_to_create = [
    "Sun",
    "Mercury",
    "Venus",
    "Earth",
    "Moon",
    "Mars",
    "Jupiter",
    "Saturn",
    "Uranus",
    "Neptune",
]

accelerations = {
    "Sun": [
        propagation_setup.acceleration.point_mass_gravity(),
        propagation_setup.acceleration.relativistic_correction(True),
    ],
    "Mercury": [propagation_setup.acceleration.point_mass_gravity()],
    "Venus": [propagation_setup.acceleration.point_mass_gravity()],
    "Earth": [propagation_setup.acceleration.point_mass_gravity()],
    "Moon": [propagation_setup.acceleration.point_mass_gravity()],
    "Mars": [propagation_setup.acceleration.point_mass_gravity()],
    "Jupiter": [propagation_setup.acceleration.point_mass_gravity()],
    "Saturn": [propagation_setup.acceleration.point_mass_gravity()],
    "Uranus": [propagation_setup.acceleration.point_mass_gravity()],
    "Neptune": [propagation_setup.acceleration.point_mass_gravity()],
}



In [5]:
global_frame_origin = "Sun"
global_frame_orientation = "J2000"
body_settings = environment_setup.get_default_body_settings(
    bodies_to_create, global_frame_origin, global_frame_orientation
)

# Create system of bodies
bodies = environment_setup.create_system_of_bodies(body_settings)

print(bodies.list_of_bodies())

['Venus', 'Uranus', 'Saturn', 'Sun', 'Mercury', 'Moon', 'Mars', 'Jupiter', 'Neptune', 'Earth']


In [6]:
observation_collection, links = batch.to_tudat(bodies=bodies)
observation_settings_list = list()
for link in list(links.values()):
    observation_settings_list.append(observation.angular_position(link))

In [7]:
end = batch.epoch_end
start = batch.epoch_start - 0*86400

In [8]:
acceleration_settings = {}
for body in batch.MPC_objects:
    acceleration_settings[str(body)] = accelerations

# acceleration_settings["TESS"] = allAccels

acceleration_models = propagation_setup.create_acceleration_models(
    bodies, acceleration_settings, bodies_to_propagate, central_bodies
)

In [9]:
initial_state_Eros = spice.get_body_cartesian_state_at_epoch("Eros", "Sun", "J2000", "NONE", end)

initial_state_Ceres = spice.get_body_cartesian_state_at_epoch("Ceres", "Sun", "J2000", "NONE", end)

initial_state_Vesta = spice.get_body_cartesian_state_at_epoch("Vesta", "Sun", "J2000", "NONE", end)
initial_states = np.concatenate([initial_state_Eros, initial_state_Ceres, initial_state_Vesta])

# Add random initial states
initial_states[0:3] += + np.random.rand(3)*1000e3
initial_states[6:9] += + np.random.rand(3)*1000e3
initial_states[12:15] += + np.random.rand(3)*1000e3

initial_states[3:6] += + np.random.rand(3)*10
initial_states[9:12] += + np.random.rand(3)*10
initial_states[15:18] += + np.random.rand(3)*10


In [10]:
termination_condition = propagation_setup.propagator.time_termination(start)


dt = -60000
# Create numerical integrator settings
integrator_settings = propagation_setup.integrator.runge_kutta_variable_step_size(
    end, dt, propagation_setup.integrator.rkf_78, dt, dt, 1.0, 1.0
)

# Create propagation settings
propagator_settings = propagation_setup.propagator.translational(
    central_bodies=central_bodies,
    acceleration_models=acceleration_models,
    bodies_to_integrate=bodies_to_propagate,
    initial_states=initial_states,
    initial_time=end,
    integrator_settings=integrator_settings,
    termination_settings=termination_condition,
)

In [11]:
# Setup parameters settings to propagate the state transition matrix
parameter_settings = estimation_setup.parameter.initial_states(
    propagator_settings, bodies
)

# Create the parameters that will be estimated
parameters_to_estimate = estimation_setup.create_parameter_set(
    parameter_settings, bodies, propagator_settings
)

print(central_bodies)
print(bodies_to_propagate)

['Sun', 'Sun', 'Sun']
['433', '1', '4']


In [12]:

estimator = numerical_simulation.Estimator(
    bodies=bodies,
    estimated_parameters=parameters_to_estimate,
    observation_settings=observation_settings_list,
    propagator_settings=propagator_settings,
    integrate_on_creation=True, 
)


pod_input = estimation.EstimationInput(
    observations_and_times=observation_collection,
    convergence_checker=estimation.estimation_convergence_checker(
        maximum_iterations=4,
    )
)

In [13]:
pod_input.define_estimation_settings(reintegrate_variational_equations=True)


In [14]:
pod_output = estimator.perform_estimation(pod_input)
results = parameters_to_estimate.parameter_vector
results = pod_output.parameter_history[:, -1]

print(results)
result_eros = results[0:6]
result_ceres = results[6:12]
result_vesta = results[12:18]



Calculating residuals and partials 3204
[ 1.00870570e+11 -2.20771328e+11 -1.07133472e+11  1.84364363e+04
  4.31828597e+03  5.78322300e+03 -3.66436947e+11 -1.29950894e+11
  1.33023730e+10  4.45852374e+03 -1.63554883e+04 -8.62158920e+03
  2.79849301e+11  2.47560320e+11  6.20525651e+10 -1.12318677e+04
  1.25412163e+04  6.47083248e+03]
Parameter update-7.10902e+07 -1.08414e+08  1.38195e+08     -4.46511    -0.919232     -15.5194      -651340 -1.15076e+06 -1.02997e+06     -1.17533     -3.24246    -0.168321      -909070       192666      -193179     -4.10406     -3.98597     -8.55003
Current residual: 0.0390229
Calculating residuals and partials 3204
Parameter update 7.10428e+07  1.08075e+08 -1.38711e+08   -0.0398548     -7.57895      9.02933       305425     -35250.6     -70356.7    0.0142112  -0.00164471   -0.0140875      -518836       100386     -27905.7   -0.0279727   -0.0136406   0.00484046
Current residual: 0.00155778
Calculating residuals and partials 3204
Parameter update    -44831.3 

In [15]:
print(f"Eros radial error: {np.sqrt(np.square((np.array(result_eros) - initial_state_Eros)[0:3]).sum())/1000} km")
print(f"Ceres radial error: {np.sqrt(np.square((np.array(result_ceres) - initial_state_Ceres)[0:3]).sum())/1000} km")
print(f"Vesta radial error: {np.sqrt(np.square((np.array(result_vesta) - initial_state_Vesta)[0:3]).sum())/1000} km")


Eros radial error: 97.3328428577204 km
Ceres radial error: 693.0867707860078 km
Vesta radial error: 1070.7573300325018 km
